# Sample Code

## 基礎建設

In [1]:
import pandas as pd
import gzip, json

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

## 載入資料

In [2]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz

--2022-01-02 10:12:37--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15499476 (15M) [application/octet-stream]
Saving to: ‘All_Beauty.csv.3’

All_Beauty.csv.3    100%[===================>]  14.78M  19.8MB/s    in 0.7s    

2022-01-02 10:12:38 (19.8 MB/s) - ‘All_Beauty.csv.3’ saved [15499476/15499476]

--2022-01-02 10:12:38--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10329961 (9.9M) [application/octet-stream]
Saving to: ‘meta_All_Beauty.json.gz.3’

meta_All_Beauty.jso 100%[===================>]   9.85M  14.0MB/s    in 0.7s   

In [3]:
metadata = getDF('/content/meta_All_Beauty.json.gz')
ratings = pd.read_csv('/content/All_Beauty.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'], header=None)

In [4]:
metadata.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,[],,[Loud 'N Clear Personal Sound Amplifier allows...,,Loud 'N Clear&trade; Personal Sound Amplifier,[],,idea village,[],"2,938,573 in Beauty & Personal Care (",[],{'ASIN: ': '6546546450'},All Beauty,,,,6546546450,[],[]
1,[],,[No7 Lift & Luminate Triple Action Serum 50ml ...,,No7 Lift &amp; Luminate Triple Action Serum 50...,"[B01E7LCSL6, B008X5RVME]",,,[],"872,854 in Beauty & Personal Care (",[],"{'Shipping Weight:': '0.3 ounces (', 'ASIN: ':...",All Beauty,"class=""a-bordered a-horizontal-stripes a-spa...",,$44.99,7178680776,[],[]
2,[],,[No7 Stay Perfect Foundation now stays perfect...,,No7 Stay Perfect Foundation Cool Vanilla by No7,[],,No7,[],"956,696 in Beauty & Personal Care (","[B01B8BR0O8, B01B8BR0NO, B014MHXXM8]","{'Shipping Weight:': '3.5 ounces (', 'ASIN: ':...",All Beauty,,,$28.76,7250468162,[],[]
3,[],,[],,Wella Koleston Perfect Hair Colour 44/44 Mediu...,[B0041PBXX8],,,[],"1,870,258 in Beauty & Personal Care (",[],"{' Item Weight: ': '1.76 ounces', 'Sh...",All Beauty,,,,7367905066,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,[],,[Lacto Calamine Skin Balance Daily Nourishing ...,,Lacto Calamine Skin Balance Oil control 120 ml...,[],,Pirmal Healthcare,[],"67,701 in Beauty & Personal Care (","[3254895630, B007VL1D9S, B00EH9A0RI, B0773MBG4...","{'Shipping Weight:': '12 ounces (', 'ASIN: ': ...",All Beauty,,,$12.15,7414204790,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [5]:
ratings.head()

,asin,reviewerID,overall,unixReviewTime
0,0143026860,A1V6B6TNIC10QE,1.0,1424304000
1,0143026860,A2F5GHSXFQ0W6J,4.0,1418860800
2,0143026860,A1572GUYS7DGSR,4.0,1407628800
3,0143026860,A1PSGLFK1NSVO,5.0,1362960000
4,0143026860,A6IKXKZMTKGSC,5.0,1324771200


## 資料整理

In [4]:
ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')

## 資料切分

In [5]:
ratings_trainings = ratings[
    (ratings['DATE'] < '2018-09-01')
]
ratings_testings = ratings[
    (ratings['DATE'] >= '2018-09-01') & 
    (ratings['DATE'] <= '2018-09-30')
]
ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
users = list(ratings_testings_by_user.keys())

## EDA

In [43]:
print('Size of testing data:', len(users))

Size of testing data: 584


In [44]:
print('Number of users who have purchase records in training data:', len(set(users).intersection(ratings_trainings['reviewerID'])))

Number of users who have purchase records in training data: 38


In [45]:
38/584

0.06506849315068493

Only 6% of testing users have purchase record in training data. 
Content-based recommender system seems not to be a good choice.

## 產生推薦

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# The dtype of "description" is list, concat the elements and make them become str
str_description = metadata['description'].apply(lambda x: ' '.join(x))

# Instead of using "title", use "asin" to get unique item metadata
df = metadata.drop_duplicates('asin')

tf = TfidfVectorizer(analyzer='word')
tfidf_matrix = tf.fit_transform(str_description)
similarity_matrix = cosine_similarity(tfidf_matrix)
mapping = pd.Series(df.index,index = df['asin'])

In [20]:
# 每個商品回傳 k 個最相近的商品
def recommend_item(item_input, k=2):
    try:
        item_index = mapping[item_input]
        similarity_score = list(enumerate(similarity_matrix[item_index]))
        similarity_score = sorted(similarity_score, key=lambda x: x[1], reverse=True)
        similarity_score = similarity_score[:k]
        item_indices = [i[0] for i in similarity_score]
        return (df['asin'].iloc[item_indices].tolist())
    except:
        return []

# 利用使用者購買過的商品產生推薦
def recommend_items(items, k):
    res = []
    for d in items:
        res.extend(recommend_item(d, k))
    return res

In [33]:
def recommender(training_data, users=[], k=10):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    recommendations = {}
    '''
    content-based
    '''

    ratings_trainings = training_data
    for user in users:
        # items purchased by the user
        purchased = ratings_trainings[ratings_trainings['reviewerID'] == user]['asin'].tolist()

        # Make sure the result of recommend_items() is unique
        recommended = list(set(recommend_items(purchased, k)))

        # Check the count of recommended; 
        # if the count is gt k, keep the top 10
        # elif the count is lt k, fill it with the most popular items
        recommended_count = len(recommended)
        if recommended_count > k:
            recommended = recommended[:10]
        elif recommended_count < k:
            most_popular = ratings['asin'].value_counts().index[:k].to_list()
            short = k - recommended_count
            recommended.extend(most_popular[:short])
        
        recommendations[user] = recommended

    return recommendations

ratings_by_user = recommender(ratings_trainings, users)
ratings_by_user

{'A100XQFWKQ30O2': ['B000FOI48G',
  'B000GLRREU',
  '1620213982',
  'B001QY8QXM',
  'B01DKQAXC0',
  'B00W259T7G',
  'B006IB5T4W',
  'B00005JS5C',
  'B0012Y0ZG2',
  'B000WYJTZG'],
 'A103T1QOGFCSEH': ['B000FOI48G',
  'B000GLRREU',
  '1620213982',
  'B001QY8QXM',
  'B01DKQAXC0',
  'B00W259T7G',
  'B006IB5T4W',
  'B00005JS5C',
  'B0012Y0ZG2',
  'B000WYJTZG'],
 'A106UKKSJ2KXPF': ['B000FOI48G',
  'B000GLRREU',
  '1620213982',
  'B001QY8QXM',
  'B01DKQAXC0',
  'B00W259T7G',
  'B006IB5T4W',
  'B00005JS5C',
  'B0012Y0ZG2',
  'B000WYJTZG'],
 'A10A7GV4D5A11V': ['B000FOI48G',
  'B000GLRREU',
  '1620213982',
  'B001QY8QXM',
  'B01DKQAXC0',
  'B00W259T7G',
  'B006IB5T4W',
  'B00005JS5C',
  'B0012Y0ZG2',
  'B000WYJTZG'],
 'A1119JJ37ZLB8R': ['B000FOI48G',
  'B000GLRREU',
  '1620213982',
  'B001QY8QXM',
  'B01DKQAXC0',
  'B00W259T7G',
  'B006IB5T4W',
  'B00005JS5C',
  'B0012Y0ZG2',
  'B000WYJTZG'],
 'A113UOOLBSZN52': ['B000FOI48G',
  'B000GLRREU',
  '1620213982',
  'B001QY8QXM',
  'B01DKQAXC0',
  'B00W

## 結果評估

In [34]:
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))

    score = total / len(ratings_testings)
    return score

evaluate(ratings_testings_by_user, ratings_by_user)

0.08305084745762711